In [9]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from urllib.parse import quote
import datetime
import seaborn as sns
from datetime import datetime, timedelta
import datetime
import matplotlib.pyplot as plt
import itertools
# import statsmodels.api as sm
# from pylab import rcParams

import warnings; 
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore",category=UserWarning)
warnings.simplefilter(action="ignore",category=FutureWarning)
warnings.filterwarnings("ignore")

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['grid.linestyle'] = ':'   
plt.rcParams['axes.grid'] = True

sns.set_style("whitegrid", {'axes.grid' : False})
#sns.color_palette("RdBu", n_colors=10)

np.float_ = np.float64

# %matplotlib inline
# %config InlineBackend.figure_formats = {'png', 'retina'}


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import math
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen


print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)
print('Seaborn version', sns.__version__)

Numpy version 2.0.1
Pandas version 2.2.2
Seaborn version 0.13.2


In [10]:
cur_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("current times:", cur_time)

current times: 2024-08-05 13:01:10


In [11]:
df = pd.read_csv("./data/dataset.csv", encoding="cp874", )

In [12]:
df.loc[df['RACE'] == 'ไทย', 'RACE'] = '1'
df.loc[df['RACE'] == 'จีน', 'RACE'] = '2'
df.loc[df['RACE'] == 'ฮ๋องกง', 'RACE'] = '2'
df.loc[df['RACE'] == 'ไต้หวัน', 'RACE'] = '2'
df.loc[df['RACE'] == 'พม่า', 'RACE'] = '3'
df.loc[df['RACE'] == 'มาเลเซีย', 'RACE'] = '4'
df.loc[df['RACE'] == 'กัมพูชา', 'RACE'] = '5'
df.loc[df['RACE'] == 'ลาว', 'RACE'] = '6'
df.loc[df['RACE'] == 'เวียดนาม', 'RACE'] = '7'
df.loc[df['RACE'] == 'อื่นๆ', 'RACE'] = '8'

In [13]:
df['NADDRCODE'] = df.ADDRCODE.apply(lambda x: x[:4])
# df

,E0,E1,PE0,PE1,DISEASE,HN,NMEPAT,SEX,AGEY,AGEM,...,DATEDEFINE,DATEDEATH,DATERECORD,DATEREACH,INTIME,ORGANISM,COMPLICA,ICD10,OFFICEID,NADDRCODE
0,942.0,4,191.0,3.0,26,000491517,NaN,1,31,3,...,9/1/2019,NaN,10/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9602
1,952.0,14,197.0,5.0,26,00056933,NaN,1,62,1,...,9/1/2019,NaN,9/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601
2,964.0,26,29.0,1.0,26,1791,NaN,2,39,6,...,6/1/2019,NaN,6/1/2019,7/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9602
3,966.0,28,196.0,4.0,26,000233446,NaN,2,32,4,...,9/1/2019,NaN,9/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601
4,954.0,16,96.0,2.0,26,000231638,NaN,1,15,0,...,6/1/2019,NaN,8/1/2019,8/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5216,22471.0,1218,NaN,NaN,26,NaN,NaN,2,2,9,...,20/12/2023,NaN,12/1/2024,12/1/2024,NaN,NaN,NaN,NaN,96010120,9601
5217,22472.0,1219,NaN,NaN,26,NaN,NaN,2,43,2,...,20/12/2023,NaN,12/1/2024,12/1/2024,NaN,NaN,NaN,NaN,96010120,9601
5218,22473.0,1220,NaN,NaN,26,NaN,NaN,1,51,0,...,23/12/2023,NaN,12/1/2024,12/1/2024,NaN,NaN,NaN,NaN,96010120,9606
5219,22474.0,1221,NaN,NaN,26,NaN,NaN,1,14,5,...,24/12/2023,NaN,12/1/2024,12/1/2024,NaN,NaN,NaN,NaN,96010120,9601


In [14]:
# Strip any leading/trailing whitespace from the date strings
df['DATESICK'] = df['DATESICK'].str.strip()

# Convert 'DATESICK' column to datetime
df['DATESICK'] = pd.to_datetime(df['DATESICK'], format='%d/%m/%Y')

In [15]:
new_df = df[~df["NADDRCODE"].str.contains('96')]
# new_df

,E0,E1,PE0,PE1,DISEASE,HN,NMEPAT,SEX,AGEY,AGEM,...,DATEDEFINE,DATEDEATH,DATERECORD,DATEREACH,INTIME,ORGANISM,COMPLICA,ICD10,OFFICEID,NADDRCODE
2969,16834.0,192,3854.0,22.0,66,000165930,NaN,1,41,3,...,13/10/2022,NaN,14/10/2022,15/10/2022,True,NaN,NaN,A90,96010000,6503


In [16]:
df = df[df["NADDRCODE"].str.contains('96')]

In [17]:
# df.head()

,E0,E1,PE0,PE1,DISEASE,HN,NMEPAT,SEX,AGEY,AGEM,...,DATEDEFINE,DATEDEATH,DATERECORD,DATEREACH,INTIME,ORGANISM,COMPLICA,ICD10,OFFICEID,NADDRCODE
0,942.0,4,191.0,3.0,26,000491517,NaN,1,31,3,...,9/1/2019,NaN,10/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9602
1,952.0,14,197.0,5.0,26,00056933,NaN,1,62,1,...,9/1/2019,NaN,9/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601
2,964.0,26,29.0,1.0,26,1791,NaN,2,39,6,...,6/1/2019,NaN,6/1/2019,7/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9602
3,966.0,28,196.0,4.0,26,000233446,NaN,2,32,4,...,9/1/2019,NaN,9/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601
4,954.0,16,96.0,2.0,26,000231638,NaN,1,15,0,...,6/1/2019,NaN,8/1/2019,8/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601


In [18]:
df['case'] = 1

In [19]:
# df.head()

,E0,E1,PE0,PE1,DISEASE,HN,NMEPAT,SEX,AGEY,AGEM,...,DATEDEATH,DATERECORD,DATEREACH,INTIME,ORGANISM,COMPLICA,ICD10,OFFICEID,NADDRCODE,case
0,942.0,4,191.0,3.0,26,000491517,NaN,1,31,3,...,NaN,10/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9602,1
1,952.0,14,197.0,5.0,26,00056933,NaN,1,62,1,...,NaN,9/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601,1
2,964.0,26,29.0,1.0,26,1791,NaN,2,39,6,...,NaN,6/1/2019,7/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9602,1
3,966.0,28,196.0,4.0,26,000233446,NaN,2,32,4,...,NaN,9/1/2019,11/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601,1
4,954.0,16,96.0,2.0,26,000231638,NaN,1,15,0,...,NaN,8/1/2019,8/1/2019,False,NaN,NaN,ไม่ระบุ,96010000,9601,1


In [20]:
ndf = df[['NADDRCODE','DATESICK','case']]

In [21]:
# Group by the relevant column(s), ensure that 'date' is part of the index

ndf.set_index(['NADDRCODE', 'DATESICK'], inplace=True)

In [22]:
# Resetting just the date index to work with it directly
df_case = ndf.groupby(level='NADDRCODE').apply(
    lambda x: x.droplevel('NADDRCODE').resample('W-mon').sum().interpolate(method='linear')
)

In [23]:
df_case = df_case.reset_index()

In [24]:
# df_case.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3367 entries, 0 to 3366
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   NADDRCODE  3367 non-null   object        
 1   DATESICK   3367 non-null   datetime64[ns]
 2   case       3367 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 79.0+ KB


In [25]:
df_case.rename(columns={'NADDRCODE': "station","DATESICK":"date", "case" : "total_case"}, inplace=True)

In [26]:
DIALECT = "mysql"
SQL_DRIVER = "pymysql"
USERNAME = "user"
PASSWORD = "user"
HOST = "dengue-db"
PORT = 3306
DBNAME = "dengue"

conn_str = DIALECT + "+" + SQL_DRIVER + "://" + USERNAME + ":" +quote(PASSWORD) + "@" + HOST + ":" +str(PORT) + "/" + DBNAME

In [ ]:
with sa.create_engine(conn_str).connect() as con:
  df_case.to_sql("cases",con,index=None, if_exists='replace')

In [ ]:
print('upload cases sucess')

In [16]:
df_case.to_csv(r"./data/new_case_p.csv", index=False)